In [5]:
import pythreejs
from ipywidgets import Text, Output, VBox, embed
from IPython.display import display
import numpy as np
import matplotlib
import matplotlib.pylab as plt
from tacs import TACS

# Load the data
loader = TACS.FH5Loader()
loader.loadData('output64.f5')

# Determine which elements to mask out based on the design variable value
upper = 0.5
val_mask = loader.computeValueMask(TACS.HEXA_ELEMENT, 'dv1', upper=upper)

# Determine which elements to mask based on centroid location
base = [15.0, 5.0, 5.0]
normal = [1.0, 2.0, -2.0]
pos_mask1 = loader.computePlanarMask(TACS.HEXA_ELEMENT, base, normal)

base = [17.0, 5.0, 5.0]
normal = [1.0, 0.0, 0.0]
pos_mask2 = loader.computePlanarMask(TACS.HEXA_ELEMENT, base, normal)

# Combine the mask by finding those elements that are masked by both 
mask = val_mask*(pos_mask1 + pos_mask2)

# Compute the visible edges and faces for visualization
pts, edges, tris, values = loader.getUnmatchedEdgesAndFaces(TACS.HEXA_ELEMENT,
                                                            name='dv1', mask=mask)

# Create the color map for the visible part of the topology
cmap = plt.cm.get_cmap('YlGn')
colors = cmap(values)
colors = np.array(colors[:,:3], dtype=np.float32)
face_colors = pythreejs.BufferAttribute(colors)

# Create the nodes - shared between triangles and edges
node_pos = pythreejs.BufferAttribute(pts.reshape(-1,3))

# Create the faces for visualization
face_indices = pythreejs.BufferAttribute(np.array(tris, dtype=np.uint32))
face_buff = pythreejs.BufferGeometry(attributes={'index': face_indices,
                                                 'position': node_pos,
                                                 'color': face_colors})
face_mat = pythreejs.MeshLambertMaterial(side='DoubleSide',
                                         vertexColors='VertexColors')
face_objs = pythreejs.Mesh(face_buff, face_mat)

# Create the edges for visualization
edge_indices = pythreejs.BufferAttribute(np.array(edges, dtype=np.uint32))
line_segments = pythreejs.BufferGeometry(attributes={'index': edge_indices,
                                                     'position': node_pos})
line_mat = pythreejs.LineBasicMaterial(color='black')
line_objs = pythreejs.LineSegments(line_segments, line_mat)

# Set the width and height of the view area
view_width = 950
view_height = 400

# Set the target for the camera
target = [25.0, 5.0, 5.0]
camera = pythreejs.PerspectiveCamera(position=[25, 25, 25],
                                     aspect=view_width/view_height)

# Set up both an ambient light
ambient_light = pythreejs.AmbientLight()
key_light = pythreejs.DirectionalLight(position=[-25, -25, -25], intensity=0.7)

# Create the scene and include the objects and light
objs = [line_objs, face_objs, ambient_light, key_light]
scene = pythreejs.Scene(children=objs, background='#72645b')

# Set an orbit controller 
controller = pythreejs.OrbitControls(controlling=camera, target=target,
                                     minPolarAngle=np.pi/4, maxPolarAngle=3*np.pi/4)

# Render the result
renderer = pythreejs.Renderer(camera=camera, antialias=True,
                              scene=scene, controls=[controller],
                              width=view_width, height=view_height)

embed.embed_minimal_html('export.html', views=renderer)

renderer


Renderer(camera=PerspectiveCamera(aspect=2.375, position=(25.0, 25.0, 25.0), quaternion=(0.0, 0.0, 0.0, 1.0), …